# Belegaufgabe zum Blockkurs Python
## Simulation mit vtk
Ziel ist es ein Modul zu schreiben, welches eine Windmühle simuliert. Die Windmühle soll dabei nur
aus primitiven Objekten zusammengesetzt werden und einen Rotor besitzen der sich während der
Simulation dreht. Außerdem soll eine kleine GUI geschrieben werden (mit PyQt), worüber sich die
Simulation starten lässt und sich entsprechende Parameter (Rotationsgeschwindigkeit, Höhe, etc.)
einstellen lassen! Genutzt werden sollen die in der Übung vorgestellten vtk-Objektwrapperklassen!

------
The aim is to write a module that simulates a windmill. The windmill should only be made up of primitive objects and have a rotor that rotates during the simulation. A small GUI should also be written (with PyQt) through which the simulation can be started and the relevant parameters (rotation speed, height, etc.) can be set! The vtk object wrapper classes presented in the exercise should be used!



# Todo
- [ ] class for objects
- [ ] assemle windmill
- [ ] add GUI
- [ ] vtk ???
- [ ] what are wrappers?

In [20]:
#import vtk


material_dict = {}

def add_material(name: str, material_properties: tuple):
    if name not in material_dict:
        material_dict[name] = material_properties
    else: 
        raise Exception("Such material already exists")
    
def get_material_list():
    return material_dict.keys()


add_material("Steel", ("grey", 600, 90))  # colour, how strong it is, how it heats
add_material("Wood", ("brown", 100, 30))
add_material("Glass", ("transparent", 20, 40))
add_material("Concrete", ("grey", 200, 20))

print(get_material_list())

class Structure:
    def __init__(self, name: str, mass: float, material: str, *dimensions: float):
        self._name = name
        self._mass = mass
        if material not in material_dict:
            raise Exception("No such material known")
        else:
            self._material = material
        self._dim = []
        if len(dimensions) != 2:
            raise ValueError("Need exactly 2 dimensions")
        else:
            self._dim = sorted(dimensions)

    def __str__(self):
        return "This is a %s %s of %d kg mass, made of %s. The size is %dx%d" % (self.__class__.__name__, self._name, self._mass, self._material, self._dim[0], self._dim[1])

structure_list = []

class Rotor(Structure):
    def __init__(self,  name: str, mass: float, material: str, max_rpm: float, model: str,  efficiency: float, *dimensions: float):
        super().__init__(name, mass, material, *dimensions)
        self._max_rpm = max_rpm
        self._model = model
        self._efficiency = efficiency

rotor_list = []

class Wing(Structure):
        def __init__(self,  name: str, mass: float, material: str, number_of_wings: int, *dimensions: float):
            super().__init__(name, mass, material, *dimensions)
            self._now = number_of_wings

wing_list = []



dict_keys(['Steel', 'Wood', 'Glass', 'Concrete'])


There are objects:

- Fundament (name, mass, width, length, material)
- Tube (name, mass, diameter, heigth, material)
- Rotor (name, mass, max_rpm, width=length, height, efficiency, model)
- Wing (name, mass, number_of_wings, material, length, width)
---
therefore 
- class Structure(name, mass, dimensions, material) can be used for everything ->
- Rotor is an ingeritage of sructure with max_rpm, model and efficiency parameter
- Wing is an ingeritage of structure with number_of_wings parameter

In [21]:

Fundament = Structure("Fundament", 1000.0, "Concrete", 20, 30)
structure_list.append(Fundament)

print(Fundament)
    
Rotor1 = Rotor("Rotor", 420, "Steel", 1400, "mk. 1", 0.6, 40, 30)   
rotor_list.append(Fundament)

print(Rotor1) 

Tube1 = Structure("Tube", 650.0, "Steel", 2, 60)
structure_list.append(Tube1)

print(Tube1)

This is a Structure Fundament of 1000 kg mass, made of Concrete. The size is 20x30
This is a Rotor Rotor of 420 kg mass, made of Steel. The size is 30x40
This is a Structure Tube of 650 kg mass, made of Steel. The size is 2x60
